In [ ]:
import numpy as np
from math import pow
import scipy.optimize

## 1 载入原始数据

### 获取基站个数sta_num（第一行），终端个数dev_num（第二行），数据维度（第三行）

In [ ]:
handle_list=[]
with open('./Dataset_2016c/sample_case001_input.txt','r') as handle:
    for line in handle:
        handle_list.append(line.strip('\n'))
sta_num = int(handle_list[0])
dev_num = int(handle_list[1])
dimension = int(handle_list[2])

### 构建基站位置列表

In [ ]:
sta_list = []
for i in range(3,3+sta_num):
    sta_list.append([float(x) for x in handle_list[i].split('\t') if x != ''])

### 构建终端TOA列表

In [ ]:
toa_list = []
for i in range(3+sta_num,3+sta_num+dev_num):
    toa_list.append([float(x) for x in handle_list[i].split('\t') if x != ''])

## 2 定位基本模型

### 构建矩阵A、X、Y，并用最小二乘法求解最小值
![矩阵](Dataset_2016c/tutorial_img/matrix.png)

In [ ]:
def residual(p,mat_A,mat_Y):
    mat_temp = np.dot(mat_A,[p[0],p[1],p[2]])-mat_Y
    # delta = np.sum(np.multiply(mat_temp,mat_temp))
    delta = np.dot(mat_temp.T,mat_temp)
    return delta

handle_list=[]
with open('./Dataset_2016c/sample_case001_ans.txt','r') as handle:
    for line in handle:
        handle_list.append(line.strip('\n'))
ans_list = []
for line in handle_list:
    ans_list.append([float(x) for x in line.split('\t') if x != ''])

wave_spread_speed = 3e8
for k_2 in range(dev_num):
    mat_A,mat_Y = [],[]
    for k in range(sta_num):
        L_k = wave_spread_speed * toa_list[k_2][k]
        if L_k <= 200:
            x_k,y_k,z_k = sta_list[k][0],sta_list[k][1],sta_list[k][2]
            K_k = x_k*x_k+y_k*y_k+z_k*z_k
            break
    if not (x_k and y_k and z_k):
        print("{}:Bad signal!".format(k_2))
        continue
    for i in range(sta_num):
        L_i = wave_spread_speed * toa_list[k_2][i]
        if L_i <= float(200/.85): # 假设每个基站的通信半径为200米，考虑到NLOS，除以折减系数（超过范围虽然有测量数据，但无效）
            if L_i > 200:
                L_i *= .85
            x_i,y_i,z_i = sta_list[i][0],sta_list[i][1],sta_list[i][2]
            mat_A.append([x_i-x_k,y_i-y_k,z_i-z_k])
            K_i = x_i*x_i+y_i*y_i+z_i*z_i    
            mat_Y.append(float(K_i-L_i*L_i-K_k+L_k*L_k))
    if len(mat_Y) <4:
        print("{}:Not enough valid TOA data! Available: {}".format(k_2,len(mat_Y)))
        continue
    # 以TOA最小值对应的基站坐标为初始值
    p_init = sta_list[toa_list[k_2].index(min(toa_list[k_2]))]
    # print(min(toa_list[k_2]),toa_list[k_2].index(min(toa_list[k_2])),p)
    cons = ({'type': 'ineq', 'fun': lambda x: 40000-pow(x[0]-p_init[0],2)-pow(x[1]-p_init[1],2)-pow(x[2]-p_init[2],2)})
    param=scipy.optimize.minimize(residual,p_init,args=(mat_A,mat_Y),method='SLSQP',constraints=cons)
    print("{}:{} ({}); Answer: {}".format(k_2,np.round(param.x,2),len(mat_Y),ans_list[k_2]))